# Day 1

## Part 1
Figure out the most calories carried by any elf.

In [1]:
; TODO stream process line by by line rather than read in the entire file into memory
(let ((lines (uiop:read-file-lines "input_day_1.txt"))
      ; temporary accumulator for each elfs calories
      (cal_ct 0)
      ; variable to track the maximum calories per elf
      (max_cal 0))
    (dolist (line lines)
        ; elves are demarcated by empty lines
        (if (string= "" line)
            (progn (when (> cal_ct max_cal) (setq max_cal cal_ct)) (setq cal_ct 0))
            (incf cal_ct (parse-integer line))
        )
    )
 ; return the maximum number of calories found on an elf
 max_cal)
    

68923

## Part 2

Find the sum of the calories for the top 3 elves. 

In [2]:
(let (
      (lines (uiop:read-file-lines "input_day_1.txt"))
      (cal_ct 0)
      (calories nil))
    (dolist (line lines)
        (if (string= "" line)
            (progn (push cal_ct calories) (setq cal_ct 0))
            (progn (incf cal_ct (parse-integer line)))
        )
    )
    (let ((sorted (sort calories #'>)))
         (+ (first sorted) (second sorted) (third sorted)))
)


200044

# Day 2

## Part 1

Score games of rock-paper-scissors. We'll try first with a brute force if-else chain.

In [3]:
(defun score_selected_shape (b)
     (if (string= b "X")
         ; 1 point for playing rock
         1
         (if (string= b "Y")
             ; 2 points for playing paper
             2
             (if (string= b "Z")
                 ; 3 points for playing scissors
                 3))))

(defun score_rock_paper_scissors (a b)
    ; they played rock
    (if (string= a "A")
        ; we played rock -> tie
        (if (string= b "X")
            3
            ; we played paper -> we won
            (if (string= b "Y")
                6
                ; we played scissors -> we lost
                (if (string= b "Z")
                    0)))
    ; they played paper
    (if (string= a "B")
        ; we played rock -> we lose
        (if (string= b "X")
            0
            ; we played paper -> tie
            (if (string= b "Y")
                3
                ; we played scissors -> we win
                (if (string= b "Z")
                    6)))
    ; they played scissors
    (if (string= a "C")
        ; we played rock -> we win
        (if (string= b "X")
            6
            ; we played paper -> we lose
            (if (string= b "Y")
                0
                ; we played scissors -> tie
                (if (string= b "Z")
                    3)))))))

(defun score_game (a b)
    "Score a single game of rock-paper-scissors. Opponent plays `a` from {A,B,C}` we play `b` from {X,Y,Z}"
    (+ (score_selected_shape b) (score_rock_paper_scissors a b)))
    
(let ((lines (uiop:read-file-lines "input_day_2.txt")))
     (loop for line in lines sum (score_game (string (char line 0)) (string (char line 2)))))

SCORE_SELECTED_SHAPE

SCORE_ROCK_PAPER_SCISSORS

SCORE_GAME

11475

Now let's try association lists.

In [4]:
(let ((lines (uiop:read-file-lines "input_day_2.txt"))
      (shape_score '(("X" . 1)("Y" . 2) ("Z" . 3)))
      (game_score '(("A X" . 3)
                    ("A Y" . 6)
                    ("A Z" . 0)
                    ("B X" . 0)
                    ("B Y" . 3)
                    ("B Z" . 6)
                    ("C X" . 6)
                    ("C Y" . 0)
                    ("C Z" . 3))))
    (loop for line in lines
          for tmp_shape_score = (cdr (assoc (char line 2) shape_score :test #'string=))
          for tmp_game_score = (cdr (assoc line game_score :test #'string=))
          sum (+ tmp_shape_score tmp_game_score)))

11475

## Part 2

Score games of rock paper scissors again.

In [5]:
(let ((lines (uiop:read-file-lines "input_day_2.txt"))
      (shape_score '(("X" . 1)("Y" . 2) ("Z" . 3)))
      (game_score '(("A X" . 3)
                    ("A Y" . 6)
                    ("A Z" . 0)
                    ("B X" . 0)
                    ("B Y" . 3)
                    ("B Z" . 6)
                    ("C X" . 6)
                    ("C Y" . 0)
                    ("C Z" . 3)))
      (what_play  '(("A X" . "Z") ;; they chose rock, we need to lose so choose scissors
                    ("A Y" . "X")
                    ("A Z" . "Y")
                    ("B X" . "X") ;; they chose paper, we need to lose so choose rock
                    ("B Y" . "Y")
                    ("B Z" . "Z")
                    ("C X" . "Y") ;; they chose scissors, we need to lose so choose paper
                    ("C Y" . "Z")
                    ("C Z" . "X"))))
    (loop for line in lines
          for played = (cdr (assoc line what_play :test #'string=))
          for tmp_shape_score = (cdr (assoc played shape_score :test #'string=))
          for new_line = (replace line played :start1 2)
          for tmp_game_score = (cdr (assoc new_line game_score :test #'string=))
          sum (+ tmp_shape_score tmp_game_score)))


16862

# Day 3

## Part 1

Sum of priorities of items in both compartments.

In [6]:
;; alist mapping characters to integer priority
(defparameter priorities (concatenate 'list
                   (loop for i from 1 to 26 collect (cons (code-char (+ 96 i)) i))
                   (loop for i from 1 to 26 collect (cons (code-char (+ 64 i)) (+ 26 i)))))

(defun find_item_in_both (contents)
    "Find character that is in both the first half and second half of the `contents` string."
    (let ((first_half (subseq contents 0 (/ (length contents) 2)))
          (second_half (subseq contents (/ (length contents) 2))))
         ;; assume there is only one so can take first item in intersection of characters
         (first (intersection (coerce first_half 'list) (coerce second_half 'list))))
)

(let ((lines (uiop:read-file-lines "input_day_3.txt")))
     (loop for line in lines
           sum (cdr (assoc (find_item_in_both line) priorities)))
)

PRIORITIES

FIND_ITEM_IN_BOTH

7903

In [7]:
(let ((lines (uiop:read-file-lines "input_day_3.txt")))
    ;; step by groups of 3
     (loop for x upto (- (/ (length lines) 3) 1)
           for group = (subseq lines (* 3 x) (+ (* 3 x) 3))
           ;; do (print (reduce #'intersection (mapcar (lambda (x) (coerce x 'list)) group))))
           sum (cdr (assoc (first (reduce #'intersection (mapcar (lambda (x) (coerce x 'list)) group))) priorities)))
)

2548

`intersection` assumes set lists where "A set doesn’t contain twice the same element and is unordered". ([cookbook intersection of lists](https://lispcookbook.github.io/cl-cookbook/data-structures.html#intersection-of-lists)). If we have repeated elements in the list then we get multiple results


In [8]:
(reduce #'intersection (mapcar (lambda (x) (coerce x 'list)) (list "hlLwDGLNnTrbNdtbdb" "ZfsvJbsJTrsttHTR" "vjPQJPFcmZjVFZFvbbVcJZJPBBhLMnDBznnGMDLMwBhlmBBl")))

(#\b #\b #\b)

# Day 4

## Part 1

Calculate number of complete overlaps.

In [9]:
(ql:quickload "cl-ppcre")

To load "cl-ppcre":
  Load 1 ASDF system:
    cl-ppcre
; Loading "cl-ppcre"
..

("cl-ppcre")

In [10]:
(let ((lines (uiop:read-file-lines "input_day_4.txt")))
     (loop for line in lines
           count (ppcre:register-groups-bind ((#'parse-integer a b c d))
            ("(\\d+)-(\\d+),(\\d+)-(\\d+)" line)
            ;; assumes ordered a,b and c,d
            (or (and (>= a c) (<= b d)) (and (>= c a) (<= d b))))
    )
)

588

In [11]:
(let ((lines (uiop:read-file-lines "input_day_4.txt")))
     (loop for line in lines
           count (ppcre:register-groups-bind ((#'parse-integer a b c d))
            ("(\\d+)-(\\d+),(\\d+)-(\\d+)" line)
            ;; check whether any limit lies within the other bounds
            (or (and (>= a c) (<= a d)) (and (>= b c) (<= b d)) (and (>= c a) (<= c b)) (and (>= d a) (<= d b))))
    )
)

911

# Day 5

## Part 1

Rearange crates. Puzzle input:

```
    [H]         [D]     [P]        
[W] [B]         [C] [Z] [D]        
[T] [J]     [T] [J] [D] [J]        
[H] [Z]     [H] [H] [W] [S]     [M]
[P] [F] [R] [P] [Z] [F] [W]     [F]
[J] [V] [T] [N] [F] [G] [Z] [S] [S]
[C] [R] [P] [S] [V] [M] [V] [D] [Z]
[F] [G] [H] [Z] [N] [P] [M] [N] [D]
 1   2   3   4   5   6   7   8   9 
```

In [12]:
(defparameter crates (list
                      (list "W" "T" "H" "P" "J" "C" "F" )
                      (list "H" "B" "J" "Z" "F" "V" "R" "G")
                      (list "R" "T" "P" "H")
                      (list "T" "H" "P" "N" "S" "Z")
                      (list "D" "C" "J" "H" "Z" "F" "V" "N")
                      (list "Z" "D" "W" "F" "G" "M" "P")
                      (list "P" "D" "J" "S" "W" "Z" "V" "M")
                      (list "S" "D" "N")
                      (list "M" "F" "S" "Z" "D")))

(let ((lines (uiop:read-file-lines "input_day_5.txt")))
     (loop for line in lines
           do (ppcre:register-groups-bind ((#'parse-integer to_move from_stack to_stack))
            ("move (\\d+) from (\\d+) to (\\d+)" line)
            (let ((tmp_stack nil))
                 (progn
                     (loop repeat to_move do (push (pop (elt crates (1- from_stack))) tmp_stack))
                     (loop repeat to_move do (push (pop tmp_stack) (elt crates (1- to_stack))))
                )))))
(loop for stack in crates do (format t "~a" (first stack)))

CRATES

NIL

NIL

ZFSJBPRFP

# Day 6

## Part 1

Find the start of frame marker with 4 unique characters. 

In [13]:
(defparameter datastream (uiop:read-file-string "input_day_6.txt"))
;; (defparameter datastream "mjqjpqmgbljsphdztnvjfqwrcgsmlb")

DATASTREAM

In [14]:
(loop for i from 4 to (1- (length datastream))
      when (> (length (remove-duplicates (subseq datastream (- i 4) i) )) 3) return i)

1848

## Part 2

Start of message with 14 unique characters. Let's make a more generic function.

In [15]:
(defun find_sof (ds len)
    (loop for i from len to (1- (length ds))
      when (> (length (remove-duplicates (subseq ds (- i len) i) )) (1- len)) return i))
    

FIND_SOF

In [16]:
(find_sof datastream 14)

2308

# Day 7

## Part 1

Walk a directory structure given a series of terminal inputs and outputs. We'll need a few more pieces here:
1. a tree structure with `folder` and `file` objects; the tree could be a recursive nested list of lists where the nodes are directories and the files are leaf nodes
2. a `folder` has a `name`, `parent` and `children`
3. a `file` has a `name`, `parent`, and `size` 
4. a state machine to parse the input and build the tree

In [17]:
(defclass folder ()
    ((name :initarg :name :accessor name)
     (parent :initarg :parent :initform nil :accessor parent)
     (children :initform nil :accessor children)))

(defclass file ()
    ((name :initarg :name :accessor name)
     (parent :initarg :parent :accessor parent)
     (size :initarg :size :accessor size)))

;; add a child to a folder
(defmethod add-child ((target folder) (child file))
    (setf (parent child) target)
    (push child (children target)))
(defmethod add-child ((target folder) (child folder))
    (setf (parent child) target)
    (push child (children target)))

;; get child by name
(defmethod get-child-by-name ((target folder) (child-name string))
    (find-if #'(lambda (x) (string= (name x) child-name)) (children target)))

;; directory size
(defmethod size ((dir folder))
    (reduce '+ (children dir) :key (lambda (x) (size x))))

#<STANDARD-CLASS COMMON-LISP-USER::FOLDER>

#<STANDARD-CLASS COMMON-LISP-USER::FILE>

#<STANDARD-METHOD COMMON-LISP-USER::ADD-CHILD (FOLDER FILE) {7005753623}>

#<STANDARD-METHOD COMMON-LISP-USER::ADD-CHILD (FOLDER FOLDER) {70057C1273}>

#<STANDARD-METHOD COMMON-LISP-USER::GET-CHILD-BY-NAME (FOLDER STRING) {70058697C3}>

#<STANDARD-METHOD COMMON-LISP-USER::SIZE (FOLDER) {70058DC7F3}>

In [18]:
(ql:quickload "cl-ppcre")

To load "cl-ppcre":
  Load 1 ASDF system:
    cl-ppcre


("cl-ppcre")

; Loading "cl-ppcre"



In [19]:
(defclass terminal-parser ()
    ((current-directory :initform nil :accessor pwd)))

(defmethod parse-line ((tp terminal-parser) (line string))
    (cond
        ;; list the folder contents - nothing for us to do but would be nice to track state
        ((string= line "$ ls")
         ;; (format t "~&Listing folder contents")
         )
        ;; change directory commands
        ((ppcre:register-groups-bind (cd_to) ("\\$ cd ([\\w\\/\\.]+)" line)
            (cond 
                ((string= cd_to "/")
                     ;; (format t "~&Moving to root")
                     (setf (pwd tp) root))
                ((string= cd_to "..")
                     ;; (format t "~&Moving up to parent")
                     (setf (pwd tp) (parent (pwd tp))))
                (t
                     ;; (format t "~&Moving to folder ~a" cd_to)
                     (setf (pwd tp) (get-child-by-name (pwd tp) cd_to))))))
        ;; a directory listing
        ((ppcre:register-groups-bind (folder_name) ("dir (\\w+)" line)
             ;; (format t "~&Adding folder named ~a to children of ~a" folder_name (name (pwd tp)))
             (add-child (pwd tp) (make-instance 'folder :name folder_name))))         
        
        ;; a file listing
        ((ppcre:register-groups-bind (file_size file_name) ("(\\d+) ([\\w\\.]+)" line)
             ;; (format t "~&Adding file named ~a with size ~a to children of ~a" file_name file_size (name (pwd tp)))
             (add-child (pwd tp) (make-instance 'file :name file_name :size (parse-integer file_size)))))         
        
    ))
            

#<STANDARD-CLASS COMMON-LISP-USER::TERMINAL-PARSER>

#<STANDARD-METHOD COMMON-LISP-USER::PARSE-LINE (TERMINAL-PARSER STRING) {7005DF7D53}>

SB-INT:TYPE-STYLE-WARNING: This is not a STRING:
 NIL
See also:
  The SBCL Manual, Node "Handling of Types"
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::ROOT


In [20]:
(defparameter tp (make-instance 'terminal-parser))
(defparameter root (make-instance 'folder :name "root"))

(let ((lines (uiop:read-file-lines "input_day_7.txt")))
     (loop for line in lines do (parse-line tp line)))

TP

ROOT

NIL

Recursion based approach. Since the `size` method is already recursive the rewalks the tree many times.  

In [21]:
;; depth-first walk of the file system
(defun folder-sizes (pwd)
    (labels ((rec (pwd acc)
        (loop :for item :in (children pwd)
              :when (typep item 'folder)
              :do 
                  ;; (format t "~& dir name ~a; item name: ~a" (name pwd) (name item))
                  (setf acc (append acc (rec item nil)))
                  (push (size item) acc)
                  ;; (format t "~& acc: ~a" acc) 
              :finally (return acc))))
    (rec root nil)))
(let ((sizes (folder-sizes root)))
     (loop for s in sizes
          :when (< s 100000) :sum s))

FOLDER-SIZES

1989474

SB-INT:SIMPLE-STYLE-WARNING: The variable PWD is defined but never used.


Queue based approach

In [22]:
(let ((stack (remove-if-not (lambda (x) (typep x 'folder)) (children root)))
      (sizes nil))
    (loop :while (> (length stack) 0)
        do
        (push (size (first stack)) sizes)
        (setf stack (append stack (remove-if-not (lambda (x) (typep x 'folder)) (children (first stack)))))
        (pop stack))
     
    (loop for s in sizes
          :when (< s 100000) :sum s))

1989474

## Part 2

Find which directory to delete to free up necessary space.

In [23]:
(let ((sizes (folder-sizes root))
      (disk-size 70000000)
      (space-needed 30000000))
     ;; find out how much we need to delete
     (let ((min-delete (- space-needed (- disk-size (size root)))))
     ;; should defend against not needing to delete anything
     (loop for s in (sort sizes #'<)
           :when (> s min-delete)
           :return s)))

1111607

# Day 8

## Part 1

Find how many trees are visible.



In [24]:
;; read the input into a 2D array
(defparameter input-day-8 (uiop:read-file-lines "input_day_8.txt"))
(defparameter num-rows (length input-day-8))
(defparameter num-cols (length (first input-day-8)))
(defparameter tree-heights (make-array (list num-rows num-cols) :initial-element 0))
(dotimes (i num-rows)
    (dotimes (j num-cols)
        (setf (aref tree-heights i j) (parse-integer (nth i input-day-8) :start j :end (1+ j)))))
;; tree-heights

INPUT-DAY-8

NUM-ROWS

NUM-COLS

TREE-HEIGHTS

NIL

In [25]:
;; given coordinates i,j check if the tree is visible in any direction
(defun check-visible (i j tree-heights)
    (let ((th (aref tree-heights i j))
          (visible (make-array (list num-rows num-cols) :initial-element nil)))
         ;; check for visible in each direction
         ;; blocked if same height tree
         (or
          ;; check to the north
          (check-visible-scan-to-edge (loop for k from (1- i) downto 0 collect (list k j)) th tree-heights)
          ;; check to the east
          (check-visible-scan-to-edge (loop for k from (1+ j) to (1- num-cols) collect (list i k)) th tree-heights)
          ;; check to the south
          (check-visible-scan-to-edge (loop for k from (1+ i) to (1- num-rows) collect (list k j)) th tree-heights)
          ;; check to the west
          (check-visible-scan-to-edge (loop for k from (1- j) downto 0 collect (list i k)) th tree-heights)
    )))

;; helper function given a list of coordinates that reaches and edge will check if visible
(defun check-visible-scan-to-edge (coords tree-height tree-heights)
    (loop for (i j) in coords
          ;; do (format t "~&i = ~a; j = ~a; tree-heights[i,j]  = ~a" i j (aref tree-heights i j))
    ;; greater height than something already visible then short circuit return visible
    ;;    when (and (> height (aref tree-heights i j) (aref tree-visible i j)) return t
    ;; smaller or equal height then shortcircuit return not-visible
          when (<= tree-height (aref tree-heights i j)) return nil
    ;; reach the edge then visible
          finally (return t)
    ))

;; (check-visible-scan-to-edge (loop for i from -1 downto 0 collect (list i 1)) 1 tree-heights)
;; (check-visible 0 0 tree-heights)

;; iterate through the array and count up the visible
(let ((visible-ct 0))
     (dotimes (i num-rows)
         (dotimes (j num-cols)
             (when (check-visible i j tree-heights) (incf visible-ct))))
     visible-ct)

;; nested loop with sum and count
(loop for i from 0 to (1- num-rows)
      sum (loop for j from 0 to (1- num-cols) count (check-visible i j tree-heights)))

CHECK-VISIBLE

CHECK-VISIBLE-SCAN-TO-EDGE

1715

1715

SB-INT:SIMPLE-STYLE-WARNING: The variable VISIBLE is defined but never used.
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::CHECK-VISIBLE-SCAN-TO-EDGE


## Part 2

Scenic score

In [26]:
;; given a list of coordinates scan until tree viewing is blocked to calculate viewing distance
(defun viewing-distance (coords tree-height tree-heights)
    (let ((distance 0))
        (loop for (i j) in coords
              do (incf distance)
              when (<= tree-height (aref tree-heights i j)) return distance
              finally (return distance))))
          
(defun scenic-score (i j tree-heights)
    (let ((th (aref tree-heights i j)))
    ;; take the product of viewing distance in all directions
    (*
       ;; check to the north
        (viewing-distance (loop for k from (1- i) downto 0 collect (list k j)) th tree-heights)
        ;; check to the east
        (viewing-distance (loop for k from (1+ j) to (1- num-cols) collect (list i k)) th tree-heights)
        ;; check to the south
        (viewing-distance (loop for k from (1+ i) to (1- num-rows) collect (list k j)) th tree-heights)
        ;; check to the west
        (viewing-distance (loop for k from (1- j) downto 0 collect (list i k)) th tree-heights)
     )))

;; (viewing-distance (loop for i from -1 downto 0 collect (list i 2)) 5 tree-heights)
;; (scenic-score 3 2 tree-heights)
;; nested loop with sum and count
(loop for i from 0 to (1- num-rows)
      maximize (loop for j from 0 to (1- num-cols) maximize (scenic-score i j tree-heights)))

VIEWING-DISTANCE

SCENIC-SCORE

374400

# Day 9

## Part 1

Number of positions visited by tail.

In [27]:
(defclass rope ()
    ((head :initarg :head :initform '(0 0) :accessor head)
     (tail :initarg :tail :initform '(0 0) :accessor tail)))

(defmethod print-object ((obj rope) stream)
  (print-unreadable-object (obj stream :type t)
    (format stream "head: ~a, tail: ~a" (head obj) (tail obj))))

(defmethod move-head ((r rope) (direction string))
    ;; move the head
    (cond
        ((string= direction "R") (incf (first (head r))))
        ((string= direction "L") (decf (first (head r))))
        ((string= direction "U") (incf (second (head r))))
        ((string= direction "D") (decf (second (head r))))
        )
    ;; have the tail follow
    (cond
        ;; two steps apart left/right
        ((and (= (abs (head-tail-difference-lr r)) 2) (= (head-tail-difference-ud r) 0)) (move-tail-lr r))
        ;; two steps apart up/down
        ((and (= (head-tail-difference-lr r) 0) (= (abs (head-tail-difference-ud r)) 2)) (move-tail-ud r))

        ;; need to move diagonal
        ((= (+ (abs (head-tail-difference-lr r)) (abs (head-tail-difference-ud r))) 3) (move-tail-lr r) (move-tail-ud r)))
    
    )

;; left/right distance from head to tail
(defmethod head-tail-difference-lr ((r rope))
    (- (first (head r)) (first (tail r))))

;; up/down distance from head to tail
(defmethod head-tail-difference-ud ((r rope))
    (- (second (head r)) (second (tail r))))

;; move tail 1 unit towards head in left/right direction
(defmethod move-tail-lr ((r rope))
    (incf (first (tail r)) (signum (head-tail-difference-lr r))))

;; move tail 1 unit towards head in up/down direction
(defmethod move-tail-ud ((r rope))
    (incf (second (tail r)) (signum (head-tail-difference-ud r))))

#<STANDARD-CLASS COMMON-LISP-USER::ROPE>

#<STANDARD-METHOD COMMON-LISP:PRINT-OBJECT (ROPE T) {700A06AF93}>

#<STANDARD-METHOD COMMON-LISP-USER::MOVE-HEAD (ROPE STRING) {700A280CC3}>

#<STANDARD-METHOD COMMON-LISP-USER::HEAD-TAIL-DIFFERENCE-LR (ROPE) {700A2EB323}>

#<STANDARD-METHOD COMMON-LISP-USER::HEAD-TAIL-DIFFERENCE-UD (ROPE) {700A3744A3}>

#<STANDARD-METHOD COMMON-LISP-USER::MOVE-TAIL-LR (ROPE) {700A3FA033}>

#<STANDARD-METHOD COMMON-LISP-USER::MOVE-TAIL-UD (ROPE) {700A4976E3}>

SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::HEAD-TAIL-DIFFERENCE-LR
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::HEAD-TAIL-DIFFERENCE-UD
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::MOVE-TAIL-LR
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::MOVE-TAIL-UD


In [28]:
(defparameter r (make-instance 'rope :head '(0 0) :tail '(0 0)))
(defparameter tail-visited (list (list 0 0)))
(loop for line in (uiop:read-file-lines "input_day_9.txt")
      for direction = (subseq line 0 1)
      for num-steps = (parse-integer line :start 2)
      do (loop repeat num-steps do
               (move-head r direction)
               (pushnew (copy-list (tail r)) tail-visited :test #'equal)))
(length tail-visited)

R

TAIL-VISITED

NIL

5874

## Part 2

A rope with 10 knots

In [29]:
(defclass rope ()
    ((knots :initarg :knots :accessor knots)))

(defmethod print-object ((obj rope) stream)
  (print-unreadable-object (obj stream :type t)
    (format stream "knots: ~a" (knots obj) )))

(defmethod move-head ((r rope) (direction string))
    ;; move the first knot or head
    (cond
        ((string= direction "R") (incf (first (first (knots r)))))
        ((string= direction "L") (decf (first (first (knots r)))))
        ((string= direction "U") (incf (second (first (knots r)))))
        ((string= direction "D") (decf (second (first (knots r)))))
        )
    ;; have the rest of the knots follow in order
    (loop for knot from 1 to 9 do (have-knot-follow r knot))
    )

;; have a given knot follow the one ahead of it in the chain
(defmethod have-knot-follow ((r rope) (knot integer))
    (cond
        ;; two steps apart left/right
        ((and (= (abs (knot-distance-lr r knot)) 2) (= (knot-distance-ud r knot) 0)) (move-knot-lr r knot))
        ;; two steps apart up/down
        ((and (= (knot-distance-lr r knot) 0) (= (abs (knot-distance-ud r knot)) 2)) (move-knot-ud r knot))

        ;; need to move diagonal
        ((>= (+ (abs (knot-distance-lr r knot)) (abs (knot-distance-ud r knot))) 3) (move-knot-lr r knot) (move-knot-ud r knot))
    ))

;; left/right distance to next knot in the chain
(defmethod knot-distance-lr ((r rope) (knot integer))
    (- (first (nth (1- knot) (knots r))) (first (nth knot (knots r)))))

;; up/down distance to next knot in the chain
(defmethod knot-distance-ud ((r rope) (knot integer))
    (- (second (nth (1- knot) (knots r))) (second (nth knot (knots r)))))

;; move a knot one unit left/right towards next on in the chain
(defmethod move-knot-lr ((r rope) (knot integer))
    (incf (first (nth knot (knots r))) (signum (knot-distance-lr r knot))))

;; move a knot one unit up/down towards next on in the chain
(defmethod move-knot-ud ((r rope) (knot integer))
    (incf (second (nth knot (knots r))) (signum (knot-distance-ud r knot))))


#<STANDARD-CLASS COMMON-LISP-USER::ROPE>

#<STANDARD-METHOD COMMON-LISP:PRINT-OBJECT (ROPE T) {700539FBA3}>

#<STANDARD-METHOD COMMON-LISP-USER::MOVE-HEAD (ROPE STRING) {70055C1333}>

#<STANDARD-METHOD COMMON-LISP-USER::HAVE-KNOT-FOLLOW (ROPE INTEGER) {70056DAB03}>

#<STANDARD-METHOD COMMON-LISP-USER::KNOT-DISTANCE-LR (ROPE INTEGER) {7005779283}>

#<STANDARD-METHOD COMMON-LISP-USER::KNOT-DISTANCE-UD (ROPE INTEGER) {7005826003}>

#<STANDARD-METHOD COMMON-LISP-USER::MOVE-KNOT-LR (ROPE INTEGER) {70058D95E3}>

#<STANDARD-METHOD COMMON-LISP-USER::MOVE-KNOT-UD (ROPE INTEGER) {7005983CE3}>

SB-KERNEL:REDEFINITION-WITH-DEFMETHOD: redefining PRINT-OBJECT (#<STANDARD-CLASS COMMON-LISP-USER::ROPE>
                                                                #<SB-PCL:SYSTEM-CLASS COMMON-LISP:T>) in DEFMETHOD
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::HAVE-KNOT-FOLLOW
SB-KERNEL:REDEFINITION-WITH-DEFMETHOD: redefining MOVE-HEAD (#<STANDARD-CLASS COMMON-LISP-USER::ROPE>
                                                             #<BUILT-IN-CLASS COMMON-LISP:STRING>) in DEFMETHOD
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::KNOT-DISTANCE-LR
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::KNOT-DISTANCE-UD
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::MOVE-KNOT-LR
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::MOVE-KNOT-UD


In [30]:
(defparameter r (make-instance 'rope :knots (loop repeat 10 collect (list 0 0))))
(defparameter tail-visited (list (list 0 0)))
(loop for line in (uiop:read-file-lines "input_day_9.txt")
      for direction = (subseq line 0 1)
      for num-steps = (parse-integer line :start 2)
      do (loop repeat num-steps do
               (move-head r direction)
               (pushnew (copy-list (first (last (knots r)))) tail-visited :test #'equal)))
(length tail-visited)

R

TAIL-VISITED

NIL

2467

# Day 10

## Part 1

Keep track of register values and calculate signal strength.

In [40]:
(ql:quickload "cl-ppcre")

To load "cl-ppcre":
  Load 1 ASDF system:
    cl-ppcre


("cl-ppcre")

; Loading "cl-ppcre"



In [66]:
(defparameter register-values nil)
(defparameter X 1)
(loop for line in (uiop:read-file-lines "input_day_10.txt")
      do
      (cond
          ;; single clock cycle delay
          ((string= line "noop") (push X register-values))
          ;; two clock delay and update register
          ((ppcre:register-groups-bind ((#'parse-integer to-add)) ("addx (-?\\d+)" line) (loop repeat 2 do (push X register-values)) (incf X to-add)))
      ))
(setf register-values (reverse register-values))
(loop for cycle in '(20 60 100 140 180 220)
      sum (* cycle (nth (1- cycle) register-values)))

REGISTER-VALUES

X

NIL

(1 1 1 1 6 6 27 27 11 11 11 12 12 12 12 16 16 17 17 21 21 22 22 22 26 26 17 17
 17 36 36 31 31 31 31 36 36 37 37 -1 -1 4 4 2 2 4 4 4 4 11 11 20 20 40 40 37 37
 19 19 21 21 26 26 26 26 24 24 24 24 31 31 34 34 32 32 34 34 6 6 -1 -1 4 4 4 6
 6 38 38 11 11 11 11 11 11 11 18 18 18 40 40 21 21 21 26 26 26 19 19 36 36 29
 29 29 9 9 36 36 36 20 20 0 0 1 1 1 4 4 19 19 11 11 9 9 3 3 17 17 21 21 21 21 4
 4 26 26 26 31 31 31 31 31 33 33 33 36 36 4 4 -1 -1 -1 3 3 6 6 4 4 38 38 11 11
 16 16 32 32 14 14 21 21 21 19 19 18 18 26 26 40 40 31 31 31 16 16 32 32 34 34
 -1 -1 -1 -1 -1 -1 2 2 6 6 6 7 7 11 11 12 12 12 16 16 18 18 21 21 16 16 35 35
 26 26 28 28 32 32 32 32 32 32 35 35 37 37 37)

17020

In [82]:
(loop for sprite-offset in register-values
      for pixel from 0
      for h-offset = (mod pixel 40)
      do 
      (when (= h-offset 0) (format t "~&"))
      (format t (if (<= (abs (- sprite-offset h-offset)) 1) "#" ".")))

NIL


###..#....####.####.####.#.....##..####.
#..#.#....#.......#.#....#....#..#.#....
#..#.#....###....#..###..#....#....###..
###..#....#.....#...#....#....#.##.#....
#.#..#....#....#....#....#....#..#.#....
#..#.####.####.####.#....####..###.####.